# YAML CONFIG
Мы в Karpov.Courses заводим Text-to-Speech модель для клонирования голоса: хотим добавить боту на основе ChatGPT озвучку голосом Толи. Этот бот уже активно используется как помощник в Симулятор SQL. 

Для настройки моделей используются конфиги (конфигурационный файл, config) большого уровня вложенности, параметры из которых периодически требуется устанавливать в качестве переменных окружения.

Коллеги попросили написать утилиту для парсинга конфигов в переменные окружения и обратно. Конфиги хранятся в формате YAML (Yet Another Markup Language).

Пример конфига

```yaml

F0_path: "Utils/JDC/bst.t7"
ASR_config: "Utils/ASR/config.yml"
ASR_path: "Utils/ASR/epoch_00100.pth"

preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
    win_length: 1200
    hop_length: 300

model_params:
  dim_in: 64
  style_dim: 64
  latent_dim: 16
  num_domains: 20
  max_conv_dim: 512
  n_repeat: 4
```

Пример переменных окружения:

```yaml
F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml

...

preprocess_params.sr=24000
preprocess_params.spect_params.n_fft=2048

...

model_params.dim_in=64

```

# Задание
Напишите две функции

yaml_to_env — функция принимает на вход текст конфига и возвращает текст с переменными окружения.
env_to_yaml — функция принимает на вход текст с переменными окружения и возвращает текст с конфигом иерархической структуры.

Псевдокод для парсинга конфига

Преобразуем текст конфига в словарь (дерево)
Рекурсивно обходим дерево:
    Если значение это словарь:
        Рекурсивно обходим словарь
    Иначе:
        Формируем строку переменной окружения вида "preprocess_params.sr.n_fft=2048" и добавляем к результату
Возвращает результат в виде строки

In [11]:
import yaml

def parse_to_dict(config_text: str) -> dict:
    try:
        config_dict = yaml.safe_load(config_text)
    except yaml.YAMLError as e:
        raise ValueError(f"Ошибка парсинга YAML: {e}")
    return config_dict

def parse_config(config_file: dict, prefix="") -> str:
    result = ""
    for key, val in config_file.items():
        new_prefix = prefix + key
        if isinstance(val, dict):
            result += parse_config(val, new_prefix + '.')
        else:
            result += f"{new_prefix}={val}\n"
    return result


def yaml_to_env(config_file: str) -> str:
    config_dict = parse_to_dict(config_file)
    result = parse_config(config_dict)
    
    return result


In [13]:
# Пример использования
config_text = '''
preprocess_params:
  sr:
    n_fft: 2048
    hop_length: 512
  window: "hann"
'''
print(yaml_to_env(config_text))

preprocess_params.sr.n_fft=2048
preprocess_params.sr.hop_length=512
preprocess_params.window=hann


```yaml
F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml

preprocess_params.sr=24000
preprocess_params.spect_params.n_fft=2048


model_params.dim_in=64

```

In [35]:
def set_nested_dict(config_dict: dict, keys: list, value):
    for key in keys[:-1]:
        config_dict = config_dict.setdefault(key, {})
    config_dict[keys[-1]] = int(value) if value.isdigit() else value

In [31]:
def env_to_dict(env_list: str) -> dict:
    config_dict = dict()
    lines = env_list.strip().splitlines()
    
    for line in lines:
        if '=' in line:
            key, value = line.split('=', 1)
            keys = key.split('.')
            set_nested_dict(config_dict, keys, value)
    
    return config_dict
                
def dict_to_yaml(config_dict: dict) -> str:
    return yaml.dump(config_dict, sort_keys=False, default_flow_style=False) 

In [32]:
def env_to_yaml(env_list: str) -> str:
    config_dict = env_to_dict(env_list)
    return dict_to_yaml(config_dict)

In [33]:
to_yaml = """
F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml
preprocess_params.sr=24000
preprocess_params.spect_params.n_fft=2048


model_params.dim_in=64
"""
print(env_to_yaml(to_yaml))

F0_path: Utils/JDC/bst.t7
ASR_config: Utils/ASR/config.yml
preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
model_params:
  dim_in: 64


In [34]:
# Пример использования:
env_text = """
F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml
preprocess_params.sr=24000
preprocess_params.spect_params.n_fft=2048
model_params.dim_in=64
"""

yaml_output = env_to_yaml(env_text)
print(yaml_output)

F0_path: Utils/JDC/bst.t7
ASR_config: Utils/ASR/config.yml
preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
model_params:
  dim_in: 64


In [36]:
from typing import Any

import yaml


def yaml_to_env(config_file: str) -> str:
    """Converts yaml config to env list"""

    def _yaml_to_env(config: dict, prefix: str = "") -> str:

        env_list = ""
        for key, value in config.items():
            if isinstance(value, dict):
                env_list += _yaml_to_env(value, f"{prefix}{key}.")
            else:
                env_list += f"{prefix}{key}={value}\n"

        return env_list

    config = yaml.load(config_file, Loader=yaml.FullLoader)
    return _yaml_to_env(config)


def env_to_yaml(env_list: str) -> str:
    """Converts env list to yaml config"""

    def convert_value(value: str) -> Any:
        if value.isnumeric():
            return int(value)
        if value.replace(".", "", 1).isnumeric():
            return float(value)
        if value.lower() in ["true", "false"]:
            return value.lower() == "true"
        return value

    def _env_to_yaml(env_list: str, config: dict = {}) -> dict:
        for line in env_list.splitlines():
            key, value = line.split("=")

            value = convert_value(value)

            key = key.split(".", 1)
            if len(key) == 1:
                config[key[0]] = value
            else:
                if key[0] not in config:
                    config[key[0]] = {}
                _env_to_yaml(f"{key[1]}={value}", config[key[0]])
        return config

    return yaml.dump(_env_to_yaml(env_list), default_style="", default_flow_style=False)


if __name__ == "__main__":
    # Example
    yaml_config = """
    F0_path: "Utils/JDC/bst.t7"
    ASR_config: "Utils/ASR/config.yml"
    ASR_path: "Utils/ASR/epoch_00100.pth"

    preprocess_params:
        sr: 24000
    spect_params:
        n_fft: 2048
        win_length: 1200
        hop_length: 300

    model_params:
        dim_in: 64
        style_dim: 64
        latent_dim: 16
        num_domains: 20
        max_conv_dim: 512
        n_repeat: 4
        dropout: 0.1
        use_spectral_norm: true

    """

    env_list = yaml_to_env(yaml_config)
    print(env_list)
    print("------------------")
    yaml_config = env_to_yaml(env_list)
    print(yaml_config)


F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml
ASR_path=Utils/ASR/epoch_00100.pth
preprocess_params.sr=24000
spect_params.n_fft=2048
spect_params.win_length=1200
spect_params.hop_length=300
model_params.dim_in=64
model_params.style_dim=64
model_params.latent_dim=16
model_params.num_domains=20
model_params.max_conv_dim=512
model_params.n_repeat=4
model_params.dropout=0.1
model_params.use_spectral_norm=True

------------------
ASR_config: Utils/ASR/config.yml
ASR_path: Utils/ASR/epoch_00100.pth
F0_path: Utils/JDC/bst.t7
model_params:
  dim_in: 64
  dropout: 0.1
  latent_dim: 16
  max_conv_dim: 512
  n_repeat: 4
  num_domains: 20
  style_dim: 64
  use_spectral_norm: true
preprocess_params:
  sr: 24000
spect_params:
  hop_length: 300
  n_fft: 2048
  win_length: 1200
